# <center>ĐỒ ÁN CUỐI KÌ: LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU</center>

| MSSV     | Họ và tên       | Github    |
|----------|-----------------|-----------|
| 1712575  | Hoàng Xuân Long | dralong10 |
| 19120539 | Vương Thế Khang | vt-khang  |

## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## A. THU THẬP DỮ LIỆU

Bộ dữ liệu mà nhóm chọn là bộ dữ liệu về các bộ phim hay TV shows được thu thập từ nền tảng phát video và truyền thông phổ biến nhất **Netflix**. \
Theo như mô tả về bộ dữ liệu:
> This dataset consists of tv shows and movies available on Netflix as of 2019. The dataset is collected from Flixable which is a third-party Netflix search engine. \
> Netflix is one of the most popular media and video streaming platforms. They have over 8000 movies or tv shows available on their platform, as of mid-2021, they have over 200M Subscribers globally. This tabular dataset consists of listings of all the movies and tv shows available on Netflix, along with details such as - cast, directors, ratings, release year, duration, etc.

Có vẻ như bộ dữ liệu này thu thập các bộ phim và TV shows từ giữa năm 2021 trở về trước và lần cập nhật gần đây nhất của bộ dữ liệu là **27/09/2021 (Version 5)** nên ta có thể tin tưởng về độ chính xác của bộ dữ liệu và các kết quả thống kê. Ngoài ra bộ dữ liệu trên được thu thập từ Flixable - công cụ tìm kiếm bên thứ ba của Netflix.

Các file dữ liệu này bao gồm:
- File "netflix_titles.csv": bộ dữ liệu được thu thập từ [Kaggle](https://www.kaggle.com/shivamb/netflix-shows)
- File "description.txt": mô tả ý nghĩa của các cột (file tự tạo theo mô tả các cột thuộc tính của bộ dữ liệu)

Người thu thập dữ liệu: Shivam Bansai [@shivamb](https://www.kaggle.com/shivamb) \
License: [**CC0 1.0 Universal (CC0 1.0) Public Domain Dedication**](https://creativecommons.org/publicdomain/zero/1.0/)
>The person who associated a work with this deed has dedicated the work to the public domain by waiving all of his or her rights to the work worldwide under copyright law, including all related and neighboring rights, to the extent allowed by law. You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission.

Đầu tiên, ta sẽ xem thử bộ dữ liệu trông như thế nào nhé. Đọc dữ liệu từ file "netflix_titles.csv" và lưu vào dataframe `netflix_df`.

In [2]:
netflix_df = pd.read_csv('netflix_titles.csv')
netflix_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


## B. KHÁM PHÁ VÀ TIỀN XỬ LÝ DỮ LIỆU

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
netflix_df.shape

(8807, 12)

Vậy dữ liệu có kích thước **8807 dòng x 11 cột**

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát sơ bộ dữ liệu ta thấy mỗi dòng chứa thông tin về một bộ phim, có vẻ như không có vấn đề các dòng có ý nghĩa khác nhau.

### Dữ liệu có các dòng bị lặp không?

Kiểm tra xem dữ liệu có các dòng bị lặp không và lưu kết quả vào biến `have_duplicated_row`. Biến này sẽ có giá trị True nếu dữ liệu có các dòng bị lặp và có giá trị False nếu ngược lại.

In [4]:
have_duplicated_row = all(netflix_df.duplicated())
have_duplicated_row

False

Như vậy, không có dòng nào bị lặp lại.

### Mỗi cột có ý nghĩa gì?

Xem file "description.txt"

In [5]:
col_name, col_meaning = [], []
with open('description.txt', 'r') as f:
    for line in f.readlines():
        col_name.append(line.strip().split('\t')[0])
        col_meaning.append(line.strip().split('\t')[1])
description_df = pd.DataFrame({'Column': col_name,
                              'Meaning': col_meaning}).set_index('Column')
description_df

,Meaning
Column,
show_id,Unique ID for every Movie / Tv Show
type,Identifier - A Movie or TV Show
title,Title of the Movie / Tv Show
director,Director of the Movie
cast,Actors involved in the movie / show
country,Country where the movie / show was produced
data_added,Date it was added on Netflix
release_year,Actual Release year of the movie / show
rating,TV Rating of the movie / show


### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Xem thử kiểu dữ liệu của các cột dữ liệu

In [6]:
netflix_df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

Như vậy có thể thấy:
- Cột "date_added" đang có kiểu dữ liệu `object`. Ta cần đưa cột này về dạng `datetime`.
- Cột "cast" và "listed_in" chứa các giá trị liệt kê, do đó ta sẽ đưa cột này về dạng `list`.

Ta sẽ làm điều này ở bước tiền xử lý dữ liệu.

---

## Tiền xử lý

Thay đổi kiểu dữ liệu của cột "date_added" sang kiểu dữ liệu dạng `datetime`.

In [7]:
netflix_df['date_added'] = netflix_df['date_added'].astype('datetime64')

Thay đổi kiểu dữ liệu của cột "cast" và "listed_in" sang kiểu dữ liệu dạng `list`. Do trong 2 cột này có các giá trị NaN, nên khi chuyển về dạng `list` sẽ bị lỗi. Do đó trước khi thay đổi kiểu dữ liệu, ta sẽ thay các giá trị NaN của 2 cột này thành "" (chuỗi rỗng).

In [8]:
netflix_df['cast'] = netflix_df['cast'].replace(np.nan, '').str.split(',')
netflix_df['listed_in'] = netflix_df['listed_in'].replace(np.nan, '').str.split(',')

Xem thử kiểu dữ liệu của các cột dữ liệu sau khi tiền xử lý.

In [9]:
netflix_df.dtypes

show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             int64
rating                  object
duration                object
listed_in               object
description             object
dtype: object

---

Có vẻ các cột đều có kiểu dữ liệu phù hợp. Nếu trong quá trình phân tích dữ liệu cần phải thay đổi kiểu dữ liệu của các cột thì ta sẽ quay lại tiền xử lý ở đây, tạm thời ta chấp nhận kiểu dữ liệu hiện tại của các cột.

### Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào?

Ta sẽ xếp các cột có kiểu dữ liệu `datetime` vào nhóm numerical. Như vậy, chỉ có 2 cột numeric là "date_added" và "release_year".

Với mỗi cột numerical, ta sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), giá trị nhỏ nhất (min) và giá trị lớn nhất (max). Sau đó ta sẽ lưu 3 giá trị này vào dataframe `nume_col_df` thành 3 dòng là "ratio_missing_val", "min", "max".

In [10]:
def ratio_missing_val(s):
    return s.isnull().mean()*100

nume_col_df = netflix_df[['date_added','release_year']].agg([ratio_missing_val, min, max])
nume_col_df

,date_added,release_year
ratio_missing_val,0.113546,0.0
min,2008-01-01 00:00:00,1925.0
max,2021-09-25 00:00:00,2021.0


Min và max của các cột có vẻ không có gì bất thường.

### Với mỗi cột có kiểu dữ liệu dạng phân loại (categorical), các giá trị được phân bố như thế nào?

Có 8 cột categorical là "show_id", "type", "little", "director", "country", "rating", "duration", "description".

Với mỗi cột categorical, ta sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), số lượng các giá trị khác nhau (không xét giá trị NaN) và danh sách các giá trị khác nhau (không xét giá trị NaN). Sau đó ta sẽ lưu 3 giá trị này vào dataframe `cate_col_df` thành 3 dòng là "ratio_missing_val", "num_diff_vals", "diff_vals".

In [11]:
def ratio_missing_val(s):
    return s.isna().mean()*100
def num_diff_vals(s):
    return s.nunique()
def diff_vals(s):
    return s.dropna().unique()

cate_col_df = netflix_df[['show_id','type','title','director','country','rating','duration','description']].agg([ratio_missing_val, num_diff_vals, diff_vals])
cate_col_df

,show_id,type,title,director,country,rating,duration,description
ratio_missing_val,0.0,0.0,0.0,29.908028,9.435676,0.045418,0.034064,0.0
num_diff_vals,8807,2,8807,4528,748,17,220,8775
diff_vals,"[s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11,...","[Movie, TV Show]","[Dick Johnson Is Dead, Blood & Water, Gangland...","[Kirsten Johnson, Julien Leclercq, Mike Flanag...","[United States, South Africa, India, United St...","[PG-13, TV-MA, PG, TV-14, TV-PG, TV-Y, TV-Y7, ...","[90 min, 2 Seasons, 1 Season, 91 min, 125 min,...","[As her father nears the end of his life, film..."


## C. ĐẶT CÂU HỎI

### Câu 1: ...
### Câu 2: ...
### Câu 3: ...

1. Thống kê qua từng năm xem top 5 diễn viên đóng phim nhiều nhất.

In [177]:
def col_cast_release_year(year):
    return netflix_df[netflix_df['release_year'] == year].iloc[:,4].explode().value_counts()[0:6].index
year = np.sort(netflix_df['release_year'].unique()) 
list_top5_cast = list(map(col_cast_release_year, year))
df_top5_cast = pd.DataFrame(list_top5_cast, columns = ['Year','Top1','Top2','Top3','Top4','Top5'], index = year)
df_top5_cast.tail(5)

,Year,Top1,Top2,Top3,Top4,Top5
2017,,Craig Sechler,Fred Armisen,Sunil Sukhada,Bayoumi Fouad,Kana Hanazawa
2018,,Andrea Libman,Takahiro Sakurai,Rajesh Kawa,Jigna Bharadhwaj,Rupa Bhimani
2019,,Alessandro Juliani,Kathleen Barr,Jamie Watson,Ian James Corlett,Eric Peterson
2020,,Keith Wickham,Dayci Brookshire,Rob Rackstraw,Blossom Chukwujekwu,Cory Doran
2021,,Fortune Feimster,David Spade,London Hughes,Grey Griffin,Wataru Takagi


Top các diễn viên đóng nhiều phim nhất thay đổi theo từng năm :v Lý do em ko biết 

2. Thống kê qua từng năm xem netflix có xu hướng làm phim cho đối tượng nào (là cột rating)

In [181]:
def col_rating_release_year(year):
    return netflix_df[netflix_df['release_year'] == year].loc[:,'rating'].value_counts()[0:1].index
list_top1_rating = list(map(col_rating_release_year,year))
df_top1_rating = pd.DataFrame(list_top1_rating, columns = ['Trending'], index = year)
df_top1_rating.tail(5)

,Trending
2017,TV-MA
2018,TV-MA
2019,TV-MA
2020,TV-MA
2021,TV-MA


Netflix có xu hướng làm thể loại phim có nội dung thiết kế dành riêng cho người lớn, không phù hợp với trẻ em hay thiếu niên. 
Lý do: vì đây là nền tảng stream phim trực tuyến có trả phí và đa số là khách hàng ở độ tuổi trưởng thành đã có công việc làm, vây nên netflix hướng đến đối tượng đó

## D. PHÂN TÍCH DỮ LIỆU

### Câu 1: ...

### Câu 2: ...

### Câu 3: ...

## E. NHÌN LẠI QUÁ TRÌNH LÀM ĐỒ ÁN

## F. TÀI LIỆU THAM KHẢO